# JB Little Riding Hood Generator

Here I have created a "machine" that uses API's from Open AI to generate a story  inspired by Bruno Munari's series of Little [color] Riding Hood. The user input is a photo, to which the generator matches with the closest pantone color. The name of the pantone color will be in the title of the story, namely "Little [color] Riding Hood". A text generator API writes a storyline based on a prompt, and then a DALL-E API is used to generates a series of images based on the plot. Lastly the machine creates a PDF of the story and saves it to drive.

<br>

**Way to use:**

* Set up the machine by running the cells under "Setup"
* Color match photo to pantone color
  * Download json file for pantone color: https://drive.google.com/file/d/1Sx642yB5i4smqXsA5QKrHeHpX0nQ-LNG/view *(Make sure that the file is saved in MyDrive and not in a folder)*
  * Upload a photo of choice from drive
* Generate story inspired by pantone color by running the cell under "Story generator"
* Save PDF of story to drive by running the cell under "Create and save PDF"

<br>

ENJOY!


## Setup


Setting up the machine requires installing dependencies, connecting colab to drive and retrieving and API key. You can run the following three cells without worrying about what they do.

### Install your dependencies


These commands are used to install software packages in Python, which are collections of pre-written code that can be used to add functionality to your Python projects without having to write the code yourself. When you run these commands in Google Colab, you're asking for specific tools and libraries to be downloaded and installed on your computer or in your online environment.

In [ ]:
!pip install openai tiktoken requests orjson reportlab
!pip3 install opencv-python

### Connect colab to drive


The following code connects your Google Colab notebook to your Google Drive. It's like telling your notebook, "Hey, let's use the files and folders I have stored in my Google Drive."

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Retrieve API key


To use Open AI API's we need a key. The following code sets up a connection to OpenAI's API within a Google Colab environment, using an API key stored in Colab's user data.

In [3]:
from google.colab import userdata
from openai import OpenAI
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

## Generate Story

Generate your own Little [color] Riding Hood story by following the unstructions below!

### Pantone color selection

This code is designed to identify colors within a photo and match them with the closest Pantone Color. The code returns the name and hex code of the closest pantone color and stores them in the variables pantone_color_name and pantone_color_hex. Because the pantone color names are not always the most descriptive, the code also returns the general name of the color as defined by CSS3 stored in the variable color_name. These variables are then used in the next cell where we create our story and images. To download JSON file of pantone colors, click here: https://drive.google.com/file/d/1Sx642yB5i4smqXsA5QKrHeHpX0nQ-LNG/view?usp=sharing

In [ ]:
import cv2
import json
import webcolors
import numpy as np
from sklearn.cluster import KMeans
from scipy.spatial.distance import euclidean

''' Upload a photo from drive by clicking the folder icon on the left side, "drive",
 "MyDrive", locate your photo, click on the three dots on the right side and then
 "copy path". Paste the path below: '''
img = cv2.imread("path to photo")
hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

# Choose the number of pantone colors you want to output
num_pantone_colors = 6

# Color quanitization
pixels = hsv_img.reshape((-1, 3))
pixels = np.float32(pixels)
num_colors = 1
kmeans = KMeans(n_clusters=num_colors)
kmeans.fit(pixels)
color_hsv =np.uint8(kmeans.cluster_centers_)

# Load pantone colors from Marlon
with open('/content/drive/MyDrive/pantone-colors-fixed.json') as file:
    pantone_colors = json.load(file)

# Function to find the closest Pantone color for a given HSV color
def find_closest_pantone(color_hsv, pantone_colors):
    distances = []
    for pantone in pantone_colors:
        pantone_hsv = np.array([pantone['H'], pantone['S'], pantone['V']])
        distance = euclidean(color_hsv, pantone_hsv)
        distances.append((pantone['Name'], pantone['HexCode'], distance))

    # Sort the distances list by the distance value
    distances.sort(key=lambda x: x[2])

    # Return the names and HexCodes of the six closest Pantone colors
    return [(name, hexcode) for name, hexcode, _ in distances[:num_pantone_colors]]

# Creates array of the name and hexcode of the closest pantone colors
closest_pantone_colors = find_closest_pantone(color_hsv.flatten(), pantone_colors)

# Define variables
pantone_color_name = closest_pantone_colors[0][0]
pantone_color_hex = closest_pantone_colors[0][1]

# Functions to find the closest named color to hex code
def closest_color(requested_color):
    min_colors = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_color[0]) ** 2
        gd = (g_c - requested_color[1]) ** 2
        bd = (b_c - requested_color[2]) ** 2
        min_colors[(rd + gd + bd)] = name
    return min_colors[min(min_colors.keys())]

def hex_to_color_name(hex_code):
    rgb_tuple = webcolors.hex_to_rgb(hex_code)
    color_name = closest_color(rgb_tuple)
    return color_name

# find name of color
color_name = hex_to_color_name(pantone_color_hex)

### Story generator

This code creates a story inspired by Bruno Munari's variants of Little Red Riding Hood. Here I have used a text generator API that writes a story based on the input variable called *prompt*. A DALL-E API generates an image to each paragraph as specified by the image prompt.

In [ ]:
from IPython.display import display, Image
import requests
import textwrap

title = f"Little {pantone_color_name} Riding Hood"

prompt = f"Write a creative and playful five-paragraph story inspired by Bruno Munari's variants of Little Red Riding Hood. The main character's name is called \"{title}\", and the color theme is \"{color_name}\". Include one or more side characters that match the color \"{color_name}\". Incorporate elements of simplicity, visual exploration, and tactile experiences similar to Munari's children's books."

story = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": prompt,}
  ]
)

paragraphs = story.choices[0].message.content.split('\n\n')

image_urls = []

for i, paragraph in enumerate(paragraphs):

    # Generate the image
    image = client.images.generate(
        model="dall-e-3",
        prompt= f"Create an illustration to this storyline: {paragraph[:60]}. The color in the image should correspond to the hex code \"{color_name}\". The picture should look like very simple scribbles with few details. It is important that the style is in watercolors, thin lines that are not connected to eachother, rapidly done, repetitive patterns, no shading or depth, monochrome. There should be mostly negative space and lots of air in the picture. Capture the characters and main happenings in the story",
        size="1024x1024",
        quality="standard",
        n=1,
    )

    # Display the paragraph text
    print_wrapped(f"{paragraph}\n")

    # Display the image
    image_url = image.data[0].url
    display(Image(url=image_url))

    # Append to list
    image_urls.append(image_url)

### Create and save pdf

This code is set up to create a PDF document using the ReportLab library in Python. The PDF is saved to MyDrive.

In [ ]:
from reportlab.lib.pagesizes import letter, inch
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER
from reportlab.lib import colors
from io import BytesIO
import os, datetime

# Modify title
title_modified = title.replace(" ", "_").replace("/", "_").replace("\\", "_")
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

# pdf creation and format
pdf_file_name = f"{title_modified}_{timestamp}.pdf"
pdf_output_path = "/content/drive/MyDrive"
os.makedirs(pdf_output_path, exist_ok=True)
pdf_file_path = os.path.join(pdf_output_path, pdf_file_name)

doc = SimpleDocTemplate(pdf_file_path, pagesize= (8.5 * inch, 8.5 * inch), leftMargin=5, rightMargin=5, topMargin=0, bottomMargin=0)
styles = getSampleStyleSheet()

# custom styling
custom_style = ParagraphStyle(
    'CustomStyle',
    parent=styles['Normal'],
    fontName='Courier',
    fontSize=11,
    leading=14,
    textColor=colors.black,
)

title_style = ParagraphStyle(
    'TitleStyle',
    parent=styles['Normal'],
    fontName='Courier',
    fontSize=24,
    leading=42,
    textColor=colors.black,
    alignment=TA_CENTER,
    spaceAfter=50,
)

story = []

story.append(Paragraph(title, title_style))
story.append(PageBreak())

for paragraph, image_url in zip(paragraphs, image_urls):

    # Add image (assuming you have a way to download or access the images)
    story.append(Image(image_url, width=8.5 * inch, height=450))  # Adjust size as needed
    story.append(Spacer(1, 12))

    # Add paragraph
    story.append(Paragraph(paragraph, custom_style))
    story.append(Spacer(1, 12))

    # Add a page break after each paragraph/image pair
    story.append(PageBreak())

# Build the PDF
doc.build(story)

print(f"PDF saved as {pdf_file_path}")